In [159]:
import boto3
s3 = boto3.resource('s3')

In [160]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

s3BucketName = "textract-bucket-ver1"

textract-bucket-ver1
textract-console-us-east-2-68cd81c8-bcb6-40e2-b8e1-0837d16c06f9


In [161]:
bucket = s3.Bucket(s3BucketName)

In [116]:
for my_bucket_object in s3.objects.all():
    print(my_bucket_object)

AttributeError: 's3.ServiceResource' object has no attribute 'objects'

In [95]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}

                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '
    return text


def get_table_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test},
                                       FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks = response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    table_dict = {}
    for index, table in enumerate(table_blocks):
        table_dict[index] = generate_table(table, blocks_map, index + 1)

    return table_dict


def generate_table(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    # create Pandas DataFrame object
    table = pd.DataFrame(rows).T
    return table


def main(file_name, output_name):
    table_dict = get_table_results(file_name)

    for index, table in table_dict.items():
        table.to_csv(f"{output_name}-{index}.csv")
        # show the results
        print('CSV OUTPUT FILE: ', table)


,1,2,3,4,5,6,7,8,9,10,11
1,Unsecured and business banking,£ million,£ million,£ million,£ million,£ million,£ million,£ million,£ million,£ million,£ million
2,At 1 January 2019,1476.5,(33.2),627.0,(65.3),111.0,(63.5),0.2,(0.2),"2,214.7",(162.2)
3,Changes reflected in impairment losses:,,,,,,,,,,
4,Increases due to originations,383.6,(5.0),28.5,-,8.2,-,2.3,-,422.6,(5.0)
5,Decreases due to repayments,(346.2),0.8,(187.1),2.6,(9.6),1.2,(2.4),0.1,(545.3),4.7
6,Changes in credit risk,-,62.3,-,(74.8),-,(22.3),-,-,-,(34.8)
7,Amounts written off,(0.1),-,(0.8),0.7,(104.9),68.3,-,-,(105.8),69.0
8,Transfers between stages:,(79.5),(60.9),5.4,83.5,74.1,(22.6),-,-,-,-
9,To stage 1,656.5,(75.0),(650.6),71.9,(5.9),3.1,-,-,-,-
10,To stage 2,(720.7),13.5,728.6,(18.3),(7.9),4.8,-,-,-,-


In [178]:
my_bucket_object.key

'TSB/page-9.jpg'

In [176]:
file_name.split('.')[0][-1]

'4'

In [ ]:
documentName = "TSB/page4.jpg"
if __name__ == "__main__":
    file_name = documentName
    main(file_name, "TSB")

In [114]:
my_bucket_object.key

'TSB/page9.jpg'

In [162]:
documentName

'TSB/page4.jpg'

In [182]:
s3bucket = "textract-bucket-ver1"
bucket_path = "TSB"
s3 = boto3.resource('s3')
bucket = s3.Bucket(s3bucket)
for bucket_obj in bucket.objects.filter(Prefix=bucket_path):
    file_obj = bucket_obj.key
    file_id = file_obj.split('.')[0][-1]
    print(file_id)


0
1
2
3
4
5
6
7
8
9


In [181]:
file_obj

'TSB/page-9.jpg'